In [4]:
import config

In [5]:
import spotipy
import json
import pprint
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from pyjarowinkler import distance
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.c_id,
                                                           client_secret= config.c_se))

### Getting playlists

In [6]:
# locating playlist
playlist_country = sp.user_playlist_tracks("spotify", "3RPaWvjgg1SAi8oIh09Irp",market="GB")
playlist_rock = sp.user_playlist_tracks("spotify", "7KMtxb9kh3TtVvJTdLtJ2K",market="GB")
playlist_house = sp.user_playlist_tracks("spotify", "4VXFwpNvQysB8yS6E2t2ls",market="GB")
playlist_classicals = sp.user_playlist_tracks("spotify", "1h0CEZCm6IbFTbxThn6Xcs",market="GB")
playlist_dance = sp.user_playlist_tracks("spotify", "794tEP1QrKMcaQcFmEyFst",market="GB")
playlist_hiphop = sp.user_playlist_tracks("spotify", "0agbFCQZvK4q9WR0d4Iqcj",market="GB")
playlist_pop = sp.user_playlist_tracks("spotify", "2YRe7HRKNRvXdJBp9nXFza",market="GB")
playlist_jazz = sp.user_playlist_tracks("spotify", "7g2KyiE9RI5F1fV60OIUu4",market="GB")
playlist_kpop = sp.user_playlist_tracks("spotify", "37i9dQZF1DX14fiWYoe7Oh",market="GB")
playlist_rnb = sp.user_playlist_tracks("spotify", "37i9dQZF1DX04mASjTsvf0",market="GB")
playlist_metal = sp.user_playlist_tracks("spotify", "37i9dQZF1DX9qNs32fujYe",market="GB")
playlist_techno = sp.user_playlist_tracks("spotify", "37i9dQZF1DX6J5NfMJS675",market="GB")

### Foundation for tracks

In [7]:
track=[]

In [8]:
# extracting tracks from playlist
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id,market="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

### Foundation for artists

In [9]:
artist=[]

In [10]:
# extracting artists from track
def get_artists_from_track(track):
    return [artist["name"] for artist in track["artists"]]

In [11]:
# extracting artists from playlist
def get_artists_from_playlist(playlist_id):
    tracks_from_playlist = get_playlist_tracks("spotify", playlist_id)
    return list(get_artists_from_track(track["track"]) for track in tracks_from_playlist)

### Foundation for track ids

In [12]:
track_id=[]

### Creating dataframe for songs, artists and track ids

In [13]:
# creating a dataframe
track_df = pd.DataFrame({'title':track,
                     'artist':artist,
                     'id':track_id})

In [14]:
# function to call tracks, artists and track ids from a playlist
def append_info(playlist_id):
    global track_df
    new_track_list = get_playlist_tracks("spotify", playlist_id)
    new_artist_list = get_artists_from_playlist(playlist_id)
    new_track=[]
    new_artist=[]
    new_track_id=[]
    for i in range(0, len(new_track_list)):
        new_track.append(new_track_list[i]['track']['name'])
    for i in new_artist_list:
        new_artist.append(", ".join(i))
    for i in range(0, len(new_track_list)):
        new_track_id.append(new_track_list[i]['track']['id'])
    new_df = pd.DataFrame({'title':new_track,
                          'artist':new_artist,
                          'id':new_track_id
                          })
    track_df = track_df.append(new_df)    

In [15]:
append_info("3RPaWvjgg1SAi8oIh09Irp")

In [16]:
append_info("7KMtxb9kh3TtVvJTdLtJ2K")

In [17]:
append_info("4VXFwpNvQysB8yS6E2t2ls")

In [18]:
append_info("1h0CEZCm6IbFTbxThn6Xcs")

In [19]:
append_info("794tEP1QrKMcaQcFmEyFst")

In [20]:
append_info("0agbFCQZvK4q9WR0d4Iqcj")

In [21]:
append_info("2YRe7HRKNRvXdJBp9nXFza")

In [22]:
append_info("7g2KyiE9RI5F1fV60OIUu4")

In [23]:
append_info("37i9dQZF1DX14fiWYoe7Oh")

In [24]:
append_info("37i9dQZF1DX04mASjTsvf0")

In [25]:
append_info("37i9dQZF1DX9qNs32fujYe")

In [26]:
append_info("37i9dQZF1DX6J5NfMJS675")

### Cleaning dataframe

In [27]:
# checking duplicates
track_df[track_df.id.duplicated(keep=False)].sort_values(by =['id'])

,title,artist,id
81,Mr. Brightside,The Killers,003vvx7Niy0yvhvHt4a68B
21,Mr. Brightside,The Killers,003vvx7Niy0yvhvHt4a68B
275,Grits,RZA,00762tXbSj2w2bQMVydJZn
2693,Grits,RZA,00762tXbSj2w2bQMVydJZn
937,Ain't No Future In Yo' Frontin',"MC Breed, DFC",00ReeHCY0FQUyuAUyPJdnk
...,...,...,...
185,If I Ruled The World (Imagine That),Nas Feat. Lauryn Hill,None
338,Makin' Moves,Kirk,None
339,Walk Thru Hell (Street Version),Madman Shawn,None
1601,Tic Toc,Lords Of The Underground,None


In [28]:
# dropping duplicates
track_df.drop_duplicates(subset=['id'],keep='first',inplace=True)

In [29]:
# dropping nulls
track_df.dropna(inplace=True)

In [30]:
# resetting index
track_df.reset_index(inplace=True)
track_df

,index,title,artist,id
0,0,Crazy - Single Version,"Patsy Cline, The Jordanaires",3zpj9dvJABiyMrmLCPw6i8
1,1,Folsom Prison Blues,"Johnny Cash, The Tennessee Two",121sjkub4p4zcFNozkKnQD
2,2,I Will Always Love You,Dolly Parton,6nRvgo4ySezQKD1D7NIqAu
3,3,Springsteen,Eric Church,05ZACkzW8YbRq3eFgIfSNB
4,4,He Stopped Loving Her Today,George Jones,6qMgx8ys4MNNxypaIvbG0i
...,...,...,...,...
8964,55,Afterhour,Carlo Ruetz,2Sh9tvtvR7rmnBpeQ7KbPu
8965,56,Latex,Monika Kruse,7Lp0NobaOnTfqcHPjVXzA6
8966,57,Whisper,Anna Reusch,0j8iHPirDSkD5nUQw9XzRF
8967,58,Mother Earth - Original Mix,Fatima Hajji,4U29PAbl99xuzW3JWSX0u0


In [31]:
# dropping index column
track_df.drop(columns=['index'],inplace=True)
track_df

,title,artist,id
0,Crazy - Single Version,"Patsy Cline, The Jordanaires",3zpj9dvJABiyMrmLCPw6i8
1,Folsom Prison Blues,"Johnny Cash, The Tennessee Two",121sjkub4p4zcFNozkKnQD
2,I Will Always Love You,Dolly Parton,6nRvgo4ySezQKD1D7NIqAu
3,Springsteen,Eric Church,05ZACkzW8YbRq3eFgIfSNB
4,He Stopped Loving Her Today,George Jones,6qMgx8ys4MNNxypaIvbG0i
...,...,...,...
8964,Afterhour,Carlo Ruetz,2Sh9tvtvR7rmnBpeQ7KbPu
8965,Latex,Monika Kruse,7Lp0NobaOnTfqcHPjVXzA6
8966,Whisper,Anna Reusch,0j8iHPirDSkD5nUQw9XzRF
8967,Mother Earth - Original Mix,Fatima Hajji,4U29PAbl99xuzW3JWSX0u0


### Creating dataframe for audio features

In [32]:
# creating a features dataframe with columns only
features_df = pd.DataFrame(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                                   'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
                                   'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
                                   'time_signature'])

In [34]:
# retrieving audio features for all tracks
for i in range(0,len(track_df)):
    uri = 'spotify:track:'+track_df['id'][i]
    my_dict = sp.audio_features(uri)[0]
    my_dict_new = { key: [my_dict[key]] for key in list(my_dict.keys()) }
    features_df_new = pd.DataFrame(my_dict_new)
    features_df = features_df.append(features_df_new)
    

In [35]:
# dropping irrelevant columns
features_df.drop(columns=['type','uri','track_href','analysis_url','duration_ms','time_signature'],inplace=True)

In [36]:
features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
0,0.471,0.021,10,-24.385,1,0.0366,0.931000,0.000001,0.1030,0.3560,107.796,3zpj9dvJABiyMrmLCPw6i8
0,0.754,0.540,5,-8.326,1,0.0266,0.687000,0.000075,0.0868,0.7900,101.738,121sjkub4p4zcFNozkKnQD
0,0.446,0.142,9,-13.498,1,0.0335,0.960000,0.001140,0.1010,0.1560,135.724,6nRvgo4ySezQKD1D7NIqAu
0,0.692,0.809,2,-5.935,1,0.0255,0.100000,0.000098,0.1850,0.9270,104.023,05ZACkzW8YbRq3eFgIfSNB
0,0.430,0.173,8,-18.597,1,0.0321,0.594000,0.000111,0.2240,0.3650,72.103,6qMgx8ys4MNNxypaIvbG0i
...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.776,0.972,1,-6.503,1,0.0421,0.001580,0.917000,0.1110,0.1970,128.017,2Sh9tvtvR7rmnBpeQ7KbPu
0,0.797,0.846,1,-7.033,0,0.0839,0.000125,0.952000,0.1190,0.0325,129.005,7Lp0NobaOnTfqcHPjVXzA6
0,0.791,0.938,10,-7.155,0,0.0518,0.000238,0.917000,0.1110,0.0665,132.004,0j8iHPirDSkD5nUQw9XzRF
0,0.575,0.998,10,-5.957,0,0.0792,0.000063,0.685000,0.2700,0.6710,134.993,4U29PAbl99xuzW3JWSX0u0


### Final dataframe

In [37]:
# merging two dataframes
final_df = track_df.merge(features_df, how='inner', on='id')

In [38]:
# checking for null values
final_df[final_df.notnull()]

,title,artist,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Crazy - Single Version,"Patsy Cline, The Jordanaires",3zpj9dvJABiyMrmLCPw6i8,0.471,0.021,10,-24.385,1,0.0366,0.931000,0.000001,0.1030,0.3560,107.796
1,Folsom Prison Blues,"Johnny Cash, The Tennessee Two",121sjkub4p4zcFNozkKnQD,0.754,0.540,5,-8.326,1,0.0266,0.687000,0.000075,0.0868,0.7900,101.738
2,I Will Always Love You,Dolly Parton,6nRvgo4ySezQKD1D7NIqAu,0.446,0.142,9,-13.498,1,0.0335,0.960000,0.001140,0.1010,0.1560,135.724
3,Springsteen,Eric Church,05ZACkzW8YbRq3eFgIfSNB,0.692,0.809,2,-5.935,1,0.0255,0.100000,0.000098,0.1850,0.9270,104.023
4,He Stopped Loving Her Today,George Jones,6qMgx8ys4MNNxypaIvbG0i,0.430,0.173,8,-18.597,1,0.0321,0.594000,0.000111,0.2240,0.3650,72.103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8964,Afterhour,Carlo Ruetz,2Sh9tvtvR7rmnBpeQ7KbPu,0.776,0.972,1,-6.503,1,0.0421,0.001580,0.917000,0.1110,0.1970,128.017
8965,Latex,Monika Kruse,7Lp0NobaOnTfqcHPjVXzA6,0.797,0.846,1,-7.033,0,0.0839,0.000125,0.952000,0.1190,0.0325,129.005
8966,Whisper,Anna Reusch,0j8iHPirDSkD5nUQw9XzRF,0.791,0.938,10,-7.155,0,0.0518,0.000238,0.917000,0.1110,0.0665,132.004
8967,Mother Earth - Original Mix,Fatima Hajji,4U29PAbl99xuzW3JWSX0u0,0.575,0.998,10,-5.957,0,0.0792,0.000063,0.685000,0.2700,0.6710,134.993


In [39]:
# saving the merged dataframe to a csv
from pathlib import Path  
filepath = Path('final_df.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)
final_df.to_csv(filepath, index=False)